# IMAGE CRAWLER

In [31]:
import requests as req
from bs4 import *

In [22]:
url = "https://www.airbnb.co.uk/s/Ljubljana--Slovenia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Ljubljana%2C%20Slovenia&place_id=ChIJ0YaYlvUxZUcRIOw_ghz4AAQ&checkin=2020-11-01&checkout=2020-11-08&source=structured_search_input_header&search_type=autocomplete_click"
url2 = "http://4chan.org/"

In [23]:
def prepare_url(url):
    import re
    if re.match(r'^[\/]{2}.*', url):
        return f'http:{url}'
    elif re.match(r'^\/\w+.*', url):
        return f'{url}'
    return url

In [28]:
## function to get img links
def parse_url(url):
    ## req will send the request to this link and get the data in text format from this link webpage
    request = req.get(url)
    ## this will change text into html format
    soup = BeautifulSoup(request.text, "html.parser")
    hrefs = soup.find_all(href=True)
    ## grab all img links and append into links list
    images = soup.find_all('img')
    links = [prepare_url(link["src"]) for link in images]
    return links, hrefs
    

In [29]:
links, hrefs = parse_url(url2)
links

['http://s.4cdn.org/image/fp/logo-transparent.png',
 'http://i.4cdn.org/mu/1619588050857s.jpg',
 'http://i.4cdn.org/sp/1619599460602s.jpg',
 'http://i.4cdn.org/out/1619065380646s.jpg',
 'http://i.4cdn.org/v/1619623764401s.jpg',
 'http://i.4cdn.org/vg/1619619927336s.jpg',
 'http://i.4cdn.org/tg/1619479367032s.jpg',
 'http://i.4cdn.org/k/1619366573950s.jpg',
 'http://i.4cdn.org/int/1619637693151s.jpg']

In [30]:
href_links = []
for href in hrefs:
    href_links.append(prepare_url(href["href"]))
href_links

['http://s.4cdn.org/css/frontpage.12.css',
 'http://s.4cdn.org/image/favicon.ico',
 'http://s.4cdn.org/image/apple-touch-icon-iphone.png',
 'http://s.4cdn.org/image/apple-touch-icon-ipad.png',
 'http://s.4cdn.org/image/apple-touch-icon-iphone-retina.png',
 'http://s.4cdn.org/image/apple-touch-icon-ipad-retina.png',
 'http://www.4chan.org/',
 '#',
 'http://boards.4channel.org/a/',
 'http://boards.4channel.org/c/',
 'http://boards.4channel.org/w/',
 'http://boards.4channel.org/m/',
 'http://boards.4channel.org/cgl/',
 'http://boards.4channel.org/cm/',
 'http://boards.4chan.org/f/',
 'http://boards.4channel.org/n/',
 'http://boards.4channel.org/jp/',
 'http://boards.4channel.org/vt/',
 'http://boards.4channel.org/v/',
 'http://boards.4channel.org/vg/',
 'http://boards.4channel.org/vm/',
 'http://boards.4channel.org/vmg/',
 'http://boards.4channel.org/vp/',
 'http://boards.4channel.org/vr/',
 'http://boards.4channel.org/vrpg/',
 'http://boards.4channel.org/vst/',
 'http://boards.4channel.o